In [1]:
import ipywidgets as widgets
from IPython.display import display
from datetime import date, timedelta

import pandas_datareader as pdr
from pandas_datareader import wb
# import quandl

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import plotly.express as px
import plotly.graph_objects as go

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import requests

In [2]:
def download_oecd_gdp(country):
    # Define the URL
    url = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.NAD,DSD_NAMAIN1@DF_QNA_EXPENDITURE_NATIO_CURR,1.0/Q.Y."+country+"...B1GQ.......?startPeriod=1947-Q1"

    # Define headers to specify the desired response format
    headers = {
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers, )

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        json_data = response.json()
    else:
        print(f"Error: {response.status_code} - {response.reason}")

    currency = pd.json_normalize(json_data['structure']['attributes']['series'][4]['values'])['id']
    countries = pd.json_normalize(json_data['structure']['dimensions']['series'][2],'values')['id']
    dates = pd.json_normalize(json_data['structure']['dimensions']['observation'],'values')['start']
    headers = ['Real GDP','Nom GDP']

    data = pd.DataFrame()
    tmp = json_data['dataSets'][0]['series']
    col = 0
    for i in tmp:
        tmp_DataFrame = pd.DataFrame(tmp[i]['observations'].values())
        data[col] = list(tmp_DataFrame[0])
        col+=1

    data.index = pd.to_datetime(dates)
    data = data.rename_axis('Date')
    data.columns = pd.MultiIndex.from_product([countries, headers],names=['',''])
    return data

In [3]:
# Download Quandl
# api_key = "AZDunoHYREw9TgVdsuzS"
# df = pdr.DataReader(symbol, 'quandl', from_date, to_date, api_key=api_key)
# ticker = ['GDP','GDPC1']
# gdp_data = pdr.DataReader(['GDP','GDPC1'], 'fred', start=from_date, end=to_date)

def comp_GDPvsIndex(country, win_size, ticker, from_date, to_date):
    tick_desc = ['Nom GDP','Real GDP']

    gdp_data = download_oecd_gdp(country)
    gdp_data.sort_values(by='Date', ascending=True, na_position='last',inplace=True)    

    INDX = yf.download(ticker, from_date, to_date, interval="1mo")
    INDX = INDX.loc[:, "Adj Close":"Adj Close"]
    INDX.sort_values(by='Date', ascending=True, na_position='last',inplace=True)
    INDX.rename(columns={'Adj Close':ticker}, inplace=True)
    INDX.columns = pd.MultiIndex.from_tuples([(country, INDX.columns[0])])

    gdp_data = gdp_data.join(INDX)                   
    gdp_data = gdp_data.dropna()

    gdp_data[(country,'Nom Growth YoY')] = (gdp_data[country,'Nom GDP']/gdp_data[country,'Nom GDP'].shift(4))-1 # shifted by 4 quarters (1 year)
    gdp_data[(country,'Real Growth YoY')] = (gdp_data[country,'Real GDP']/gdp_data[country,'Real GDP'].shift(4))-1
    gdp_data[(country,ticker+' YoY')] = (gdp_data[country,ticker]/gdp_data[country,ticker].shift(4))-1

    gdp_data[(country,'Corr GDP_'+country+' vs '+ticker)] = gdp_data[country,'Real Growth YoY'].rolling(window=win_size).corr(gdp_data[country,ticker+' YoY']) 
    
    
    # Create traces
    fig = go.Figure()

    print('Average 10-yr Correlation: S&P500 YoY vs Real GDP YoY')
    lags = ['no-lag','3mo-lag','6mo-lag','9mo-lag','12mo-lag']

    for i,val in enumerate(lags):
        df = gdp_data[country,'Real Growth YoY'].rolling(window=win_size).corr(gdp_data[country,ticker+' YoY'].shift(i)).dropna()
        print(val+': \t'+str(df.mean()))
        fig.add_trace(go.Scatter(x=df.index, y=df,
                        mode='lines',
                        name=val))
    fig.update_layout(title_text="<b>GDP vs "+ticker+" Correlation<b>", title_x=0.5)    
    fig.update_xaxes(title_text="<b>Dates<b>")
    fig.update_yaxes(title_text="<b>Correlation</b>")
    fig.show()
    
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=gdp_data.index, y=gdp_data[country,'Real Growth YoY'], name="Real Growth YoY"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=gdp_data.index, y=gdp_data[country,ticker+' YoY'], name=ticker+' YoY'),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(title_text="<b>GDP vs "+ticker+"<b>", title_x=0.5)

    # Set x-axis title
    fig.update_xaxes(title_text="<b>Dates<b>")
    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Real Growth YoY</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>"+ticker+" YoY</b>", secondary_y=True)

    fig.show()
            
    return gdp_data

In [4]:
from_date = date(1900, 1, 1) 
to_date = date.today()

country = 'USA'
ticker = '^GSPC'
win_size = 40 # window in quarters e.g., 40 (10 year)


USvsSP500 = comp_GDPvsIndex(country, win_size, ticker, from_date, to_date)
USvsSP500.head(10)


ValueError: Length of values (13) does not match length of index (147)

In [ ]:
from_date = date(1900, 1, 1) 
to_date = date.today()

# FTSE - FTSE 100: Comprises the 100 largest companies listed on the London Stock Exchange (LSE). It's a key benchmark for the UK stock market.
# FTMC - FTSE 250: Tracks the 250 next largest companies after the FTSE 100 on the LSE, representing mid-cap stocks.
# FTLC - FTSE 350: Combines the FTSE 100 and FTSE 250, offering a broader representation of the UK stock market.

country = 'GBR'
ticker = '^FTLC'
win_size = 40 # window in quarters e.g., 40 (10 year)


GBRvsFTLC = comp_GDPvsIndex(country, win_size, ticker, from_date, to_date)
GBRvsFTLC.tail(10)
